In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 1. 设置中文显示
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 2. 加载数据
df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据_2.csv')

# 3. 定义关键指标
kpi_mapping = {
    '课前预习得分': '课前预学',
    '课堂互动得分': '课堂参与',
    '课后复习得分': '课后复习',
    '知识拓展得分': '延伸阅读',
    '课外自习时长': '自习时间',
    '科研实践参与得分': '实验科研时间',
    '合作学习得分': '竞赛活动时间'
}

# 提取关键指标数据
kpi_data = df[list(kpi_mapping.values())].copy()
kpi_data.columns = list(kpi_mapping.keys())

# 4. 基于学业成果的相关性分析确定权重
def determine_correlation_weights(data, outcome_variables):
    """
    使用与学业成果的相关性分析确定权重
    """
    print("=== 基于学业成果的相关性分析确定权重 ===")
    correlation_weights = {}

    for kpi in data.columns:
        # 计算与每个成果变量的平均相关性
        corrs = []
        for outcome in outcome_variables:
            # 确保两个Series长度相同，且无NaN对计算影响
            temp_data = data[[kpi]].dropna()
            temp_outcome = df[[outcome]].loc[temp_data.index].dropna() # 确保索引匹配并去除NaN
            if not temp_data.empty and not temp_outcome.empty:
                corr = np.corrcoef(temp_data[kpi], temp_outcome[outcome])[0, 1]
                if not np.isnan(corr):
                    corrs.append(abs(corr))
        if corrs:
            correlation_weights[kpi] = np.mean(corrs)
            print(f"  {kpi} 与学业成果的平均相关性: {np.mean(corrs):.3f}")
        else:
            correlation_weights[kpi] = 0 # 如果无法计算相关性，则为0
            print(f"  {kpi} 与学业成果的平均相关性: 无法计算")

    # 归一化权重
    total = sum(correlation_weights.values())
    if total == 0: # 避免除以0
        correlation_weights = {k: 1/len(data.columns) for k in data.columns} # 如果所有相关性都为0，则平均分配
    else:
        correlation_weights = {k: v/total for k, v in correlation_weights.items()}

    print("\n=== 最终权重分配 ===")
    for kpi, weight in correlation_weights.items():
        print(f"  {kpi}: {weight:.3f}")

    return {'correlation': correlation_weights}

# 5. 执行相关性权重分析
# 假设有一些学业成果变量（请根据实际数据调整）
outcome_variables = [
    '问题解决能力提升',           # 学习收获评价 - 分析问题、解决问题能力提升
    '专业课知识融合',            # 专业课评价 - 课程将知识、能力、素养进行了有机融合
    '专业课解决问题能力',        # 专业课评价 - 课程教学培养了我解决复杂问题的综合能力
    '自主学习能力提升',          # 学习收获评价 - 自主学习能力提升
    '合作能力提升',             # 学习收获评价 - 合作能力提升
    '表达沟通能力提升',         # 学习收获评价 - 书面表达和沟通能力提升
    '未来规划能力提升',         # 学习收获评价 - 有能力规划未来工作生活
    '科学精神提升',             # 核心素养提升 - 科学精神（理性思维、批判质疑、勇于探究等）
    '实践创新提升',             # 核心素养提升 - 实践创新（劳动意识、问题解决、技术应用等）
]

print("开始基于学业成果相关性的权重分析...")
scientific_weights = determine_correlation_weights(kpi_data, outcome_variables)

# 6. 使用相关性权重计算EHI
def calculate_ehi_correlation(data, weights_dict):
    """
    使用相关性分析确定的权重计算EHI
    """
    ehi_score = 0
    for kpi, weight in weights_dict.items():
        if kpi in data.columns:
            ehi_score += data[kpi] * weight * 100

    return np.clip(ehi_score, 0, 100)

# 使用相关性权重
correlation_weights = scientific_weights['correlation']
ehi_scores = calculate_ehi_correlation(kpi_data, correlation_weights)

# 7. 可视化权重分析
def visualize_correlation_weights(weights_dict, outcome_variables):
    """可视化相关性权重分析结果"""
    kpis = list(weights_dict.keys())
    weights_values = list(weights_dict.values())

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 子图1: 权重分布
    colors = ['lightblue' if x < 0.14 else 'lightgreen' for x in weights_values]
    axes[0, 0].barh(range(len(weights_dict)), weights_values, color=colors, alpha=0.7)
    axes[0, 0].set_yticks(range(len(weights_dict)))
    axes[0, 0].set_yticklabels(kpis)
    axes[0, 0].set_xlabel('权重')
    axes[0, 0].set_title('基于学业成果相关性的权重分配')
    for i, v in enumerate(weights_values):
        axes[0, 0].text(v + 0.005, i, f'{v:.3f}', va='center')
    axes[0, 0].grid(True, alpha=0.3)

    # 子图2: 各指标与学业成果的平均相关性
    avg_correlations = []
    for kpi in kpis:
        corrs = []
        for outcome in outcome_variables:
            temp_data = kpi_data[[kpi]].dropna()
            temp_outcome = df[[outcome]].loc[temp_data.index].dropna()
            if not temp_data.empty and not temp_outcome.empty:
                corr = np.corrcoef(temp_data[kpi], temp_outcome[outcome])[0, 1]
                if not np.isnan(corr):
                    corrs.append(abs(corr))
        avg_correlations.append(np.mean(corrs) if corrs else 0)

    axes[0, 1].bar(range(len(kpis)), avg_correlations, alpha=0.7, color='orange')
    axes[0, 1].set_xlabel('指标')
    axes[0, 1].set_ylabel('平均相关性')
    axes[0, 1].set_title('各指标与学业成果的平均相关性')
    axes[0, 1].set_xticks(range(len(kpis)))
    axes[0, 1].set_xticklabels(kpis, rotation=45, ha='right')
    for i, v in enumerate(avg_correlations):
        axes[0, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    axes[0, 1].grid(True, alpha=0.3)

    # 子图3: 权重与相关性的关系
    scatter = axes[1, 0].scatter(avg_correlations, weights_values, s=100, alpha=0.7)
    axes[1, 0].set_xlabel('与学业成果的平均相关性')
    axes[1, 0].set_ylabel('权重')
    axes[1, 0].set_title('权重与学业成果相关性的关系')

    # 添加标签
    for i, kpi in enumerate(kpis):
        axes[1, 0].annotate(kpi, (avg_correlations[i], weights_values[i]),
                          xytext=(5, 5), textcoords='offset points', fontsize=8)

    # 添加趋势线
    if len(set(avg_correlations)) > 1:  # 确保有变化
        z = np.polyfit(avg_correlations, weights_values, 1)
        p = np.poly1d(z)
        axes[1, 0].plot(avg_correlations, p(avg_correlations), "r--", alpha=0.8)

    correlation = np.corrcoef(avg_correlations, weights_values)[0, 1] if len(set(avg_correlations)) > 1 else 1
    axes[1, 0].text(0.05, 0.95, f'相关系数: {correlation:.3f}',
                   transform=axes[1, 0].transAxes, fontsize=10,
                   bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5))

    axes[1, 0].grid(True, alpha=0.3)

    # 子图4: 各指标对EHI的贡献
    contributions = {}
    for kpi in weights_dict.keys():
        contributions[kpi] = kpi_data[kpi].mean() * weights_dict[kpi] * 100

    contrib_series = pd.Series(contributions).sort_values(ascending=False)
    colors = ['red' if x < 8 else 'orange' if x < 12 else 'green' for x in contrib_series]

    axes[1, 1].bar(range(len(contrib_series)), contrib_series.values, color=colors, alpha=0.7)
    axes[1, 1].set_xlabel('指标')
    axes[1, 1].set_ylabel('对EHI的平均贡献')
    axes[1, 1].set_title('各指标对EHI的平均贡献')
    axes[1, 1].set_xticks(range(len(contrib_series)))
    axes[1, 1].set_xticklabels(contrib_series.index, rotation=45, ha='right')

    for i, v in enumerate(contrib_series.values):
        axes[1, 1].text(i, v + 0.3, f'{v:.1f}', ha='center', va='bottom', fontsize=9)

    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return weights_dict

# 执行可视化
final_weights = visualize_correlation_weights(correlation_weights, outcome_variables)

# 8. 输出最终权重和解释
print("\n" + "="*60)
print("基于学业成果相关性的权重分配及解释:")
print("="*60)

weight_explanations = {
    '课堂互动得分': "权重最高，课堂参与与学业成果相关性最强",
    '课前预习得分': "较高权重，预习对学习效果有显著影响",
    '课后复习得分': "较高权重，复习巩固与学业成果密切相关",
    '知识拓展得分': "中等权重，知识拓展对学业有一定贡献",
    '课外自习时长': "中等权重，自习时间与学业成果相关",
    '科研实践参与得分': "较低权重，科研实践对学业成果影响相对较小",
    '合作学习得分': "最低权重，合作学习与学业成果相关性最弱"
}

for kpi, weight in final_weights.items():
    explanation = weight_explanations.get(kpi, "需要进一步研究")
    print(f"{kpi}: {weight:.3f} - {explanation}")

# 9. 保存权重结果
weights_df = pd.DataFrame.from_dict(scientific_weights)
weights_df.to_csv('../../data/intermediate/correlation_weights_analysis.csv', encoding='utf-8-sig')

print(f"\n相关性权重分析结果已保存到: correlation_weights_analysis.csv")

# 10. 使用相关性权重重新计算EHI
df['EHI_correlation'] = ehi_scores
df['EHI_level_correlation'] = pd.cut(ehi_scores,
                                   bins=[0, 59, 79, 100],
                                   labels=['低健康度', '中等健康度', '高健康度'])

print(f"\n=== 基于相关性的EHI统计 ===")
print(f"平均EHI: {ehi_scores.mean():.2f}")
print(f"标准差: {ehi_scores.std():.2f}")
print(f"范围: [{ehi_scores.min():.2f}, {ehi_scores.max():.2f}]")
print(f"健康等级分布:")
print(df['EHI_level_correlation'].value_counts())

# 11. 保存包含新EHI分数的数据
df.to_csv('../../data/intermediate/数据_包含相关性EHI.csv', encoding='utf-8-sig', index=False)
print(f"\n包含相关性EHI分数的数据已保存")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 1. 加载数据
print("正在加载数据...")
df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据_2.csv')

# 2. 定义关键指标映射
kpi_mapping = {
    '课前预习得分': '课前预学',
    '课堂互动得分': '课堂参与',
    '课后复习得分': '课后复习',
    '知识拓展得分': '延伸阅读',
    '课外自习时长': '自习时间',
    '科研实践参与得分': '实验科研时间',
    '合作学习得分': '竞赛活动时间'
}

# 提取关键指标数据
kpi_data = df[list(kpi_mapping.values())].copy()
kpi_data.columns = list(kpi_mapping.keys())

# 3. 定义最终权重
final_weights = {
    '课前预习得分': 0.205,
    '课堂互动得分': 0.211,
    '课后复习得分': 0.207,
    '知识拓展得分': 0.192,
    '课外自习时长': 0.063,
    '科研实践参与得分': 0.051,
    '合作学习得分': 0.071
}

# 4. 计算EHI分数
print("正在计算EHI分数...")
ehi_scores = 0
for kpi, weight in final_weights.items():
    ehi_scores += kpi_data[kpi] * weight * 100

ehi_scores = np.clip(ehi_scores, 0, 100)
df['EHI_score'] = ehi_scores
df['EHI_level'] = pd.cut(ehi_scores,
                        bins=[0, 59, 79, 100],
                        labels=['低健康度', '中等健康度', '高健康度'])

# 5. 学院级别EHI分析
print("正在分析学院级别EHI指标...")
college_ehi = df.groupby('学院').agg({
    'EHI_score': ['count', 'mean', 'std', 'min', 'median', 'max'],
    'EHI_level': lambda x: (x == '高健康度').sum()  # 高健康度人数
}).round(2)

# 重命名列
college_ehi.columns = ['学生人数', 'EHI平均分', 'EHI标准差', 'EHI最低分', 'EHI中位数', 'EHI最高分', '高健康度人数']

# 计算高健康度比例和排名
college_ehi['高健康度比例'] = (college_ehi['高健康度人数'] / college_ehi['学生人数'] * 100).round(2)
college_ehi['EHI排名'] = college_ehi['EHI平均分'].rank(ascending=False).astype(int)
college_ehi = college_ehi.sort_values('EHI平均分', ascending=False)

# 6. 专业级别EHI分析
print("正在分析专业级别EHI指标...")
major_ehi = df.groupby(['学院', '专业']).agg({
    'EHI_score': ['count', 'mean', 'std', 'min', 'median', 'max'],
    'EHI_level': lambda x: (x == '高健康度').sum()  # 高健康度人数
}).round(2)

# 重命名列
major_ehi.columns = ['学生人数', 'EHI平均分', 'EHI标准差', 'EHI最低分', 'EHI中位数', 'EHI最高分', '高健康度人数']

# 计算高健康度比例和排名
major_ehi['高健康度比例'] = (major_ehi['高健康度人数'] / major_ehi['学生人数'] * 100).round(2)
major_ehi['EHI排名'] = major_ehi['EHI平均分'].rank(ascending=False).astype(int)
major_ehi = major_ehi.sort_values('EHI平均分', ascending=False)

# 7. 保存结果到新文件
print("正在保存结果...")

# 学院EHI指标
college_ehi.to_csv('../../data/processed/学院EHI指标统计.csv', encoding='utf-8-sig')
print("✓ 学院EHI指标已保存: 学院EHI指标统计.csv")

# 专业EHI指标
major_ehi.to_csv('../../data/processed/专业EHI指标统计.csv', encoding='utf-8-sig')
print("✓ 专业EHI指标已保存: 专业EHI指标统计.csv")

# 8. 生成简要报告
print("\n" + "="*50)
print("EHI指标分析简要报告")
print("="*50)
print(f"总学生数: {len(df):,}人")
print(f"总体EHI平均分: {df['EHI_score'].mean():.2f}分")
print(f"学院数量: {len(college_ehi)}个")
print(f"专业数量: {len(major_ehi)}个")
print(f"\nEHI最高学院: {college_ehi.index[0]} ({college_ehi.iloc[0]['EHI平均分']}分)")
print(f"EHI最低学院: {college_ehi.index[-1]} ({college_ehi.iloc[-1]['EHI平均分']}分)")
print(f"\nEHI最高专业: {major_ehi.index[0]} ({major_ehi.iloc[0]['EHI平均分']}分)")
print(f"EHI最低专业: {major_ehi.index[-1]} ({major_ehi.iloc[-1]['EHI平均分']}分)")

# 9. 可视化 - 学院EHI排名
plt.figure(figsize=(12, 8))
colleges = college_ehi.index
scores = college_ehi['EHI平均分']

# 使用颜色区分
colors = ['#ff6b6b' if x < 60 else '#feca57' if x < 70 else '#48dbfb' for x in scores]

bars = plt.barh(range(len(colleges)), scores, color=colors, alpha=0.8)
plt.yticks(range(len(colleges)), colleges, fontsize=10)
plt.xlabel('EHI平均分')
plt.title('各学院EHI平均分排名', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()

# 添加数值标签
for i, v in enumerate(scores):
    plt.text(v + 0.3, i, f'{v:.1f}', va='center', fontweight='bold')

plt.tight_layout()


# 10. 可视化 - 专业EHI分布
plt.figure(figsize=(10, 6))
plt.hist(major_ehi['EHI平均分'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(major_ehi['EHI平均分'].mean(), color='red', linestyle='--', label=f'平均分: {major_ehi["EHI平均分"].mean():.2f}')
plt.xlabel('专业EHI平均分')
plt.ylabel('专业数量')
plt.title('专业EHI平均分分布', fontsize=16, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()

print("\n" + "="*50)
print("分析完成！所有文件已保存到 data/results/ 目录")
print("="*50)